<a href="https://colab.research.google.com/github/shrigoa/random/blob/main/Random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem:
1. Manage internet bandwidth usage on a ship.

Important data:
1. Access to two communication links each contracted with 15TB of bandwidth for a month, thus, total of 30TB.

Goals:
1. Optimal usage: Bandwidth should be consumed only when required and minimized when not needed.
2.a. Fairness Across Time: Quality of Experience (QoE) should be consistent across different timeslots.
2.b. Significant variations — such as excellent QoE in some periods and poor QoE in others — must be inimized to avoid service quality alarms.
2.c. Demand met percentage is the QoE. It should not vary much.

Given Data:
1. Per hour demand data for upstream and downstream traffic
- Variations of 20-25% are possible
- Columns: Time slots, downstream demand, upstream demand
2. Ship arrival at ports
- Variation of 1-2 hours earlier or later.
- Demand drops 50% at port compared to sea.
- Columns: Port, Arrival Time, Departure Time
3. Link Downtime
- One of the links will be unavailable on certain days.
- This downtime must be configurable in the code.
- During downtime, the algorithm must reserve enough bandwidth on the remaining active link to maintain stable QoE and avoid service degradation.
4. Consumption Limits:
- Each link supports maximum bandwidth of 500Mbps at any given point.
- Bandwidth planning is done in 5 min time-slots.
- Bandwidth volume (e.g., 1 GB in 5 minutes) will internally be converted into rate for enforcement.
5. End of the month requirement:
- Allocated 30TB should be completely used. (CONSTRAINT)
- No under/over usage allowed.
6. Objective:
- Demand met percentage is the QoE (OBJECTIVE FUNCTION).
- Needs to be as high as possible.

7. Parameters:
- Number of 5 mins slots per day: 288 slots
- Number of slots per hour: 12 slots
- Number of days: 30 days
- Seconds in each slot:
- Downtime start day: 19
- Downtime end day: 27
- Capacity per link: 15 TB
- Number of links: 2
- Number of links up during downtime: 1
- Maximum bandwidth of any link: 500 Mbps

7. Decision Variables
- How much data is consumed in each 5 mins slot: data_consumed[day, hour, slot]

8. Objective Function
- demand[day, hour] - sum of data_consumed[day, hour, slot] over all slots for each day, for each hour data







In [1]:
#Install packages
!pip install gurobipy

In [2]:
#Import packages
import numpy as np
import pandas as pd
import math
import gurobipy as gp
from gurobipy import GRB
from datetime import datetime, timedelta

In [3]:
#Parameters
#Fixed Parameters
hours_in_day = 24
mins_in_hour = 60
secs_in_min = 60

#Configurable Parameters
slot_mins = 5
planning_start_date = datetime(2025, 5, 1)
num_planning_days = 30
active_links = 2
cap_per_link = 15000000
max_bandwidth_per_link = 500
downtime_start_day = datetime(2025, 5, 20)
downtime_end_day = datetime(2025, 5, 28)
active_links_downtime = 1

#Derived Parameters
secs_in_hour = mins_in_hour * secs_in_min
planning_end_date = planning_start_date + timedelta(days = num_planning_days - 1)
seconds_in_slot = slot_mins * secs_in_min
seconds_in_hour = mins_in_hour * secs_in_min
slots_per_hour = int(mins_in_hour/slot_mins)
slots_per_day = int(slots_per_hour * hours_in_day)

total_cap = cap_per_link * active_links
total_cap_downtime = cap_per_link * active_links_downtime

max_data_per_slot_per_link = max_bandwidth_per_link * seconds_in_slot
max_data_per_hour_per_link = max_data_per_slot_per_link * slots_per_hour

max_data_per_slot = max_data_per_slot_per_link * active_links
max_data_per_hour = max_data_per_hour_per_link * active_links

downtime_max_data_per_slot = max_data_per_slot_per_link * active_links_downtime
downtime_max_data_per_hour = max_data_per_hour_per_link * active_links_downtime

downtime_start_day_num = (downtime_start_day - planning_start_date).days
downtime_end_day_num = (downtime_end_day - planning_start_date).days

In [4]:
#Data Creation
input_data_df = pd.DataFrame(columns = ['Day', 'Hour', 'Downstream Demand', 'Upstream Demand', 'Demand Met'])
for day in range(num_planning_days):
  for hour in range(hours_in_day):
    input_data_df.loc[len(input_data_df)] = [day, hour, 62000, 0, 0]

In [5]:
#Define Model
env = gp.Env()
env.setParam('OutputFlag', 0)
mip = gp.Model('PlanningMIP', env = env)

Restricted license - for non-production use only - expires 2026-11-23


In [6]:
#Decision Variables
slot_consumption = {}
hour_consumption = {}
unmet_demand = {}

for day in range(num_planning_days):
  for hour in range(hours_in_day):
    max_available_data = downtime_max_data_per_hour if (day >= downtime_start_day_num and day <= downtime_end_day_num) else max_data_per_hour
    hour_consumption[day, hour] = mip.addVar(lb = 0, ub = max_available_data, vtype=GRB.CONTINUOUS, name = f'hour_consumption_{day}_{hour}')
    unmet_demand[day, hour] = mip.addVar(lb = 0, ub = max_available_data, vtype=GRB.CONTINUOUS, name = f'unmet_demand_{day}_{hour}')

    # for slot in range(slots_per_day):
    #   slot_consumption[day, hour, slot] = mip.addVar(lb = 0, ub = max_data_per_slot, vtype=GRB.CONTINUOUS, name = f'slot_consumption_{day}_{hour}_{slot}')

In [7]:
#Constraints

#Demand Constraint
for day in range(num_planning_days):
  for hour in range(hours_in_day):
    mip.addConstr(hour_consumption[day, hour] <= input_data_df[(input_data_df['Day'] == day) & (input_data_df['Hour'] == hour)]['Downstream Demand'], name=f'demand data')

#Total Capacity Constraint
data_consumed_exp = gp.LinExpr()
for day in range(num_planning_days):
  for hour in range(hours_in_day):
    data_consumed_exp += hour_consumption[day, hour]
mip.addConstr(data_consumed_exp == total_cap, name=f'total data consumed')

#Maximum Capacity Allowed during Downtime
downtime_data_consumed_exp = gp.LinExpr()
for day in range(downtime_start_day_num, downtime_end_day_num + 1):
  for hour in range(hours_in_day):
    downtime_data_consumed_exp += hour_consumption[day, hour]
mip.addConstr(downtime_data_consumed_exp <= total_cap_downtime, name=f'downtime data consumed')

#Unmet Demand Constraint
for day in range(num_planning_days):
  for hour in range(hours_in_day):
    mip.addLConstr(unmet_demand[day, hour] + hour_consumption[day, hour], GRB.GREATER_EQUAL, input_data_df[(input_data_df['Day'] == day) & (input_data_df['Hour'] == hour)]['Downstream Demand'], name=f'unmet demand constraint')



<ipython-input-7-d5351cbe1118>:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mip.addConstr(hour_consumption[day, hour] <= input_data_df[(input_data_df['Day'] == day) & (input_data_df['Hour'] == hour)]['Downstream Demand'], name=f'demand data')
<ipython-input-7-d5351cbe1118>:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mip.addConstr(hour_consumption[day, hour] <= input_data_df[(input_data_df['Day'] == day) & (input_data_df['Hour'] == hour)]['Downstream Demand'], name=f'demand data')
<ipython-input-7-d5351cbe1118>:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  mip.addConstr(hour_consumption[day, hour] <= input_data_df[(input_data_df['Day'] == day) & (input_data_df['Hour'] == hour)]['Down

In [8]:
#Objective
unmet_demand_exp = gp.LinExpr()
for day in range(num_planning_days):
  for hour in range(hours_in_day):
    unmet_demand_exp += unmet_demand[day, hour]
mip.setObjective(unmet_demand_exp, GRB.MINIMIZE)

In [9]:
#Solve Model
mip.write('mip.lp')
mip.optimize()

In [10]:
#Result
if mip.status == GRB.OPTIMAL:
  print("Found Optimal Solution!!")
  print("Objective", mip.ObjVal)

  counter = 0
  for day in range(num_planning_days):
    for hour in range(hours_in_day):
      input_data_df.loc[counter, "Demand Met"] = hour_consumption[day, hour].X
      counter += 1
else:
  print("Not Optimal")

Found Optimal Solution!!
Objective 14640000.0
<gurobi.Var unmet_demand_0_0 (value 8000.0)>
<gurobi.Var unmet_demand_0_1 (value 62000.0)>
<gurobi.Var unmet_demand_0_2 (value 62000.0)>
<gurobi.Var unmet_demand_0_3 (value 62000.0)>
<gurobi.Var unmet_demand_0_4 (value 62000.0)>
<gurobi.Var unmet_demand_0_5 (value 62000.0)>
<gurobi.Var unmet_demand_0_6 (value 62000.0)>
<gurobi.Var unmet_demand_0_7 (value 62000.0)>
<gurobi.Var unmet_demand_0_8 (value 62000.0)>
<gurobi.Var unmet_demand_0_9 (value 62000.0)>
<gurobi.Var unmet_demand_0_10 (value 62000.0)>
<gurobi.Var unmet_demand_0_11 (value 62000.0)>
<gurobi.Var unmet_demand_0_12 (value 62000.0)>
<gurobi.Var unmet_demand_0_13 (value 62000.0)>
<gurobi.Var unmet_demand_0_14 (value 62000.0)>
<gurobi.Var unmet_demand_0_15 (value 62000.0)>
<gurobi.Var unmet_demand_0_16 (value 62000.0)>
<gurobi.Var unmet_demand_0_17 (value 62000.0)>
<gurobi.Var unmet_demand_0_18 (value 62000.0)>
<gurobi.Var unmet_demand_0_19 (value 62000.0)>
<gurobi.Var unmet_demand_

In [11]:
#Write Output
input_data_df.to_csv("Demand and Output.csv")

In [12]:
input_data_df['Demand Met'].sum()/1000000

np.float64(30.0)

In [13]:
input_data_df['Downstream Demand'].sum()/1000000

np.float64(44.64)